In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import xarray as xr
import cmocean.cm as cmocean
import glob

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client
import matplotlib.path as mpath
import datetime as dt

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 4,Total memory: 100.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43129,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 100.00 GiB
Comm: tcp://10.6.122.15:40283,Total threads: 1
Dashboard: /proxy/46595/status,Memory: 25.00 GiB
Nanny: tcp://127.0.0.1:33729,


In [3]:
# database for control simulation:
master_session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')

In [4]:
expt='01deg_jra55v140_iaf_cycle2'

variable='geolon_t'
geolon_t = cc.querying.getvar(expt,variable,master_session, n=-1)
geolon_t.load()
print(geolon_t.shape)

variable='geolat_t'
geolat_t = cc.querying.getvar(expt,variable,master_session, n=-1)
geolat_t.load()
print(geolat_t.shape)


(2700, 3600)
(2700, 3600)


In [5]:
from cosima_cookbook import explore
dbx = explore.DatabaseExplorer #(session=session)
dbx

cosima_cookbook.explore.DatabaseExplorer

In [6]:
ee = explore.ExperimentExplorer(session=master_session, experiment='01deg_jra55v13_ryf9091')
ee

ExperimentExplorer(children=(HTML(value='\n            <h3>Experiment Explorer</h3>\n\n            <p>Select a…

In [7]:
#Frequency, experiment and session do not need to be specified if they were defined in the previous step
def getACCESSdata(var, start, end, exp, freq, ses, minlon, maxlon, minlat = -90, maxlat = -45):
    '''
    Inputs:
    var - Short name for the variable of interest
    start - Time from when data has to be returned
    end - Time until when data has to be returned
    exp - Experiment name
    freq - Time frequency of the data
    ses - Cookbook session
    minlat - minimum latitude from which to return data. If not set, defaults to min latitude in geolon_t data array.
    maxlat - maximum latitude from which to return data. If not set, defaults to max latitude in geolon_t data array.
    minlon - minimum longitude from which to return data. If not set, defaults to -90 to cover the Southern Ocean.
    maxlon - minimum longitude from which to return data. If not set, defaults to -45 to cover the Southern Ocean.

    Output:
    Data array with corrected time and coordinates within the specified time period and spatial bounding box.
    '''
    #Accessing data
    vararray = cc.querying.getvar(exp, var, ses, frequency = freq, start_time = start, end_time = end)
    #Applying time correction
    vararray['time'] = vararray.time - dt.timedelta(hours = 12)
    # assign new coordinates to SST dataset
    #.coords extracts the values of the coordinate specified in the brackets
    vararray.coords['ni'] = geolon_t['xt_ocean'].values
    vararray.coords['nj'] = geolon_t['yt_ocean'].values
    #Rename function from xarray uses dictionaries to change names. Keys refer to current names and values are the desired names
    vararray = vararray.rename(({'ni':'xt_ocean', 'nj':'yt_ocean'}))
    #Subsetting data to area of interest
    #Subsetting sea ice concentration array
    vararray = vararray.sel(xt_ocean = slice(minlon, maxlon)).sel(yt_ocean = slice(minlat, maxlat))
    return vararray

## the following loading of ice thickness works fine from the RYF expt

In [8]:
expt = '01deg_jra55v13_ryf9091'
start_time='2151-11-01'
end_time='2152-01-01'

In [9]:
hi_m_con=getACCESSdata(var='hi_m',start=start_time,end=end_time,exp=expt,freq='1 monthly',ses=master_session, minlon=geolon_t.xt_ocean.values.min(),maxlon=geolon_t.xt_ocean.values.max())
hi_m_con

<xarray.DataArray 'hi_m' (time: 3, yt_ocean: 740, xt_ocean: 3600)>
dask.array<getitem, shape=(3, 740, 3600), dtype=float32, chunksize=(1, 675, 900), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) object 2151-10-31 12:00:00 ... 2151-12-31 12:00:00
    TLON      (yt_ocean, xt_ocean) float32 dask.array<chunksize=(675, 900), meta=np.ndarray>
    TLAT      (yt_ocean, xt_ocean) float32 dask.array<chunksize=(675, 900), meta=np.ndarray>
    ULON      (yt_ocean, xt_ocean) float32 dask.array<chunksize=(675, 900), meta=np.ndarray>
    ULAT      (yt_ocean, xt_ocean) float32 dask.array<chunksize=(675, 900), meta=np.ndarray>
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -45.18 -45.11 -45.04
Attributes:
    units:          m
    long_name:      grid cell mean ice thickness
    cell_measures:  area: tarea
    cell_methods:   time: mean
    time_rep:       averaged
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 3, d2: 2)>\ndask.a...
    ncfiles:        ['/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf90...
    contact:        Andy Hogg
    email:          andy.hogg@anu.edu.au
    created:        2020-06-11
    description:    0.1 degree ACCESS-OM2 global model configuration under th...
    notes:          Additional daily outputs saved from 1 Jan 1950 to 31 Dec ...

# below are the lines that fail in the folllowing expt

In [10]:
var = cc.querying.getvar(expt, 'hi_m', master_session, frequency='1 monthly',start_time=start_time, end_time=end_time)

In [11]:
var

<xarray.DataArray 'hi_m' (time: 3, nj: 2700, ni: 3600)>
dask.array<concatenate, shape=(3, 2700, 3600), dtype=float32, chunksize=(1, 675, 900), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2151-11-01 00:00:00 ... 2152-01-01 00:00:00
    TLON     (nj, ni) float32 dask.array<chunksize=(675, 900), meta=np.ndarray>
    TLAT     (nj, ni) float32 dask.array<chunksize=(675, 900), meta=np.ndarray>
    ULON     (nj, ni) float32 dask.array<chunksize=(675, 900), meta=np.ndarray>
    ULAT     (nj, ni) float32 dask.array<chunksize=(675, 900), meta=np.ndarray>
Dimensions without coordinates: nj, ni
Attributes:
    units:          m
    long_name:      grid cell mean ice thickness
    cell_measures:  area: tarea
    cell_methods:   time: mean
    time_rep:       averaged
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 3, d2: 2)>\ndask.a...
    ncfiles:        ['/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf90...
    contact:        Andy Hogg
    email:          andy.hogg@anu.edu.au
    created:        2020-06-11
    description:    0.1 degree ACCESS-OM2 global model configuration under th...
    notes:          Additional daily outputs saved from 1 Jan 1950 to 31 Dec ...

In [12]:
print(var.time)
var['time'] = var.time - dt.timedelta(hours = 12)

<xarray.DataArray 'time' (time: 3)>
array([cftime.datetime(2151, 11, 1, 0, 0, 0, 0, calendar='noleap', has_year_zero=True),
       cftime.datetime(2151, 12, 1, 0, 0, 0, 0, calendar='noleap', has_year_zero=True),
       cftime.datetime(2152, 1, 1, 0, 0, 0, 0, calendar='noleap', has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 2151-11-01 00:00:00 ... 2152-01-01 00:00:00
Attributes:
    long_name:  model time
    bounds:     time_bounds


In [13]:
# this IAF fails to load sea ice data with same function calls

In [14]:
start_time = '2018-11-01'
end_time = '2019-01-01'
expt='01deg_jra55v140_iaf_cycle3'


In [15]:
hi_m_con= getACCESSdata(var='hi_m',start=start_time,end=end_time,exp=expt,freq='1 monthly',ses=master_session, minlon=geolon_t.xt_ocean.values.min(),maxlon=geolon_t.xt_ocean.values.max())
hi_m_con

UFuncTypeError: ufunc 'subtract' cannot use operands with types dtype('<M8[ns]') and dtype('O')

In [16]:
var = cc.querying.getvar(expt, 'hi_m', master_session, frequency='1 monthly',start_time=start_time, end_time=end_time)

In [17]:
print(var.time)
var['time'] = var.time - dt.timedelta(hours = 12)

<xarray.DataArray 'time' (time: 3)>
array(['2018-11-01T00:00:00.000000000', '2018-12-01T00:00:00.000000000',
       '2019-01-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2018-11-01 2018-12-01 2019-01-01
Attributes:
    long_name:  model time
    bounds:     time_bounds


UFuncTypeError: ufunc 'subtract' cannot use operands with types dtype('<M8[ns]') and dtype('O')